# Combination of Datasets

Pandas offers a variety of tools to handle the combination of different datasets. Anything from simple concatenations to more complex operations such as database joins and merges are supported.

In [1]:
import numpy as np
import pandas as pd

`make_df()` is a function to quickly create a `DataFrame` of a particular form, which is useful for brevity of the following examples:

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


The `display` class allows us to display multiple `DataFrame` objects side by side:

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Data Concatenation

Pandas provides us with the function `pd.concat()`, which is very similar to `np.concatenate()` but with a number of options to reflect on the final result:

In [4]:
print(pd.concat.__code__.co_varnames)

('objs', 'axis', 'join', 'ignore_index', 'keys', 'levels', 'names', 'verify_integrity', 'sort', 'copy', 'op')


Further information on what each of these parameters stand for can be found on the official documentation, or, more conveniently, using IPython's `?` operator:

In [5]:
pd.concat?

`pd.concat()` can be used to perform simple concatenation of `Series` or `DataFrame` objects, just as `np.concatenate()` can be used to concatenate arrays:

In [6]:
# Concatenate Series
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [7]:
# Concatenate DataFrame
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Concatenation takes place row-wise by default (`axis=0`) within the `DataFrame`. `np.concat()` allows specification of the axis along which concatenation will take place:

In [8]:
# Concatenate along columns
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')") # 'columns' is equivalent to 1 here

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

One thing to note about `pd.concat()` is that the operation preserves the original indices. This could lead to duplicate indices, which is often undesirable:

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


The `verify_integrity` flag can be specified to raise an exception if that does happen:

In [10]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


If the index itself is not particularly important, the `ignore_index` can be specified for Pandas to create a new integer index for the resulting `DataFrame`:

In [11]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


The `keys` option allow us to specify a label for each data source. The result is a hierarchically indexed object containing the data:

In [12]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

In practice, data from different sources might have different sets of column names, as opposed to the previous examples. `pd.concat()` offers several options for such cases and we will explore some of those. By default, Pandas will fill the entries for which no data is available with `NaN`:

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In order to obtain a different outcome, we can specify the `join` parameter. By default, the join is a union of the input columns (`join='outer'`), but this can be changed to an intersection of the columns with `join='inner'`:

In [14]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Another option is to specify the index of the remaining columns. Since `join_axes` is now deprecated, we can do:

In [15]:
display('df5', 'df6', "pd.concat([df5, df6]).reindex(df5.columns, axis=1)")

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
3,B3,C3,D3
4,B4,C4,D4
,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


Since concatenation is a very common operation, `Series` and `DataFrame` objects have the `append` method to accomplish the same thing in a more convenient way:

In [16]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Few things to keep in mind: 
* The Pandas `append()` method differs from the Python `append()` method in that it does not modify the original object directly, instead a new one is created. 
* It also is not very efficient, because it creates a new index and data buffer. Using `concat()` with a list of objects is generally better than doing multiple `append` operations.

## Data Join and Merge


The `pd.merge()` function implements three types of joins: _one-to-one_ , _many-to-one_ and _many-to-many_ joins. The type of join performed depends on the form of the input data. Below are some simple examples of the three types of merges:

### One-to-one joins

The simplest type of a merge is the one-to-one join, which is very similar to the column-wise concatenation seen above:

In [17]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Notice that `pd.merge()` correctly accounts for the fact that the order of the "employee" columns is different between `df1` and `df2`.

In [18]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### Many-to-one joins

Many-to-one joins occur when one of the two key columns contains duplicate entries. For theses cases, the result will preserve those duplicates as appropriate:

In [19]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

The resulting `DataFrame` has an additional `supervisor` column (compared with `df3`), where the data (`Guido`) is duplicated where appropriate (`Jake` and `Lisa` rows).

### Many-to-many joins

Many-to-many joins are similar to many-to-one joins, but with a twist: when the key column in both the left and right array contains duplicates, the result will account for all of those by duplicating the data where appropriate:

In [20]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In the result above we can see that, in `df5`, the group `Accounting` has two skills associated with it across two different entries (`math` and `spreadsheets`). This makes it so that for every `Accounting` entry in `df1`, there will be two almost identical new entries in the result, so that every combination between `group` and `skills` (in this case `Accounting`-`math` and `Accounting`-`spreadsheets`) is accounted for.

### Behavior specification

The default behavior of `pd.merge()` is to look for one or more matching column names between the two inputs and use this as the key. `pd.merge()` provides a variety of options to change its behavior when the default is not what we want.

#### The `on` parameter

The name of the key column can be explicitly specified by using the `on` parameter:

In [21]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Evidently this options require that both the left and right `DataFrame` objects have the specified column name.

#### The `left_on` and `right_on` parameters

For cases in which we would like to merge datasets with different column names, we can use `left_on` and `right_on` to specify the name of the columns of interest:

In [22]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has two identical and, therefore, redundant columns: `employee` and `name`. We can drop one of them by using the `drop()` method of `DataFrame`:

In [23]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### The `left_index` and `right_index` parameters

It is also possible to merge on an index rather than on a column. Consider the following data:

In [24]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


We can specify the index as the key for merging by specifying `left_index` and/or `right_index` in `pd.merge()`:

In [25]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


For the sake of convenience, `DataFrame` implements the `join()` method, which performs a merge that default to joining on indices:

In [26]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


We can go one step further by mixing indices and columns by combining `left_index` with `right_on` or `right_index` with `left_on`:

In [27]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Although the above examples did not use multiple indices or multiple columns, performing such operations will work just as well.

### Set Arithmetic for Joins

The examples above left an important consideration in performing joins behind: the type of set arithmetic used. This is a very important characteristic of the operation that will come up when a value appears in one key column but not the other. The following example better illustrates that:

In [28]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Here, the only `name` entry in common is `Mary`. By default, the result will contain the _intersection_ between the two sets of inputs (also known as _inner join_). This behavior can be explicitly specified using the `how` parameter:

In [29]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for `how` include `'outer'`, `'left'`, and `'right'`.  An _outer join_ returns the join over the union of the input columns, filling all missing values with NAs:

In [30]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

Finally, the _left join_ and _right join_ returns joins over the left and right entires, respectively:

In [31]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

The output rows now correspond to the entries in the left input (`df6`). A _right join_ works similarly.

### Overlapping column names

In cases where the two input `DataFrame`s have conflicting column names, we can use the `suffixes` parameter. Consider the following:

In [32]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Both `df8` and `df9` have a column with the exact same name (`rank`), so the merge function automatically appends a suffix (in this case `_x` and `_y`) to distinguish between the two. We can specify what these suffixes will be by using the `suffixes` parameter:

In [33]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

The book goes, in detail, over a very good example (using real data) covering many of what was discussed here. The example can be found [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html#Example:-US-States-Data).